# Graded Excersizes L4 - Piotr Bonar, Frederic Evenepoel


### Graded Exercise 1 (6 points)

In [71]:
pip install python-Levenshtein

Note: you may need to restart the kernel to use updated packages.


1. First, use the Gutenberg corpus from the NLTK package. Using the sents() function of the corpus, take 50 sentences as test and the rest for training. In these 50 test sentences, replace one word for a random in-vocabulary word and save the original sentence for comparison. Show a selection of 5 of these in the final report. You can also try to be a bit deliberate with your choice of random words to assess complex scenarios.



In [72]:
import random
import nltk
import Levenshtein
from nltk.corpus import gutenberg as corpus

nltk.download('gutenberg')
nltk.download('punkt')

sentences = corpus.sents()
words = [w for w in set(corpus.words()) if w.isalpha()]

train_sentences = [["<s>"] + sent + ["</s>"] for sent in sentences[200:]]
test_sentences = [["<s>"] + sent + ["</s>"] for sent in sentences[:50]]

words.extend(["<s>", "</s>"])

def get_similar_word(word, vocab):
    word_lower = word.lower()
    similar_words = [w for w in vocab if Levenshtein.distance(word_lower, w.lower()) == 1]
    return random.choice(similar_words) if similar_words else word

altered_sentences = []
original_sentences = []

for sentence in test_sentences:
    sentence_words = [w for w in sentence if w.isalpha()]
    if len(sentence_words) > 1:
        original_sentences.append(" ".join(sentence))

        word_to_replace = random.choice(sentence_words)
        new_word = get_similar_word(word_to_replace, words)

        altered_sentence = [new_word if w == word_to_replace else w for w in sentence]
        altered_sentences.append(" ".join(altered_sentence))

for i in range(0,7):
    print(f"Original: {original_sentences[i]}")
    print(f"Altered:  {altered_sentences[i]}\n")


[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/macbook/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /Users/macbook/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Original: <s> [ Emma by Jane Austen 1816 ] </s>
Altered:  <s> [ Emma Bay Jane Austen 1816 ] </s>

Original: <s> VOLUME I </s>
Altered:  <s> Volum I </s>

Original: <s> CHAPTER I </s>
Altered:  <s> CHAPTERS I </s>

Original: <s> Emma Woodhouse , handsome , clever , and rich , with a comfortable home and happy disposition , seemed to unite some of the best blessings of existence ; and had lived nearly twenty - one years in the world with very little to distress or vex her . </s>
Altered:  <s> Emma Woodhouse , handsome , clever , and rich , wish a comfortable home and happy disposition , seemed to unite some of the best blessings of existence ; and had lived nearly twenty - one years in the world wish very little to distress or vex her . </s>

Original: <s> She was the youngest of the two daughters of a most affectionate , indulgent father ; and had , in consequence of her sister ' s marriage , been mistress of his house from a very early period . </s>
Altered:  <s> Shew was the youngest 

2. Implement and train an 𝑛-gram language model using the NLTK package. Try using bigrams and trigrams and the variations seen in class.


In [73]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE, Laplace, StupidBackoff

n = 3
train_data_mle, vocab_mle = padded_everygram_pipeline(n, train_sentences)
train_data_lpc, vocab_lpc = padded_everygram_pipeline(n, train_sentences)
train_data_sbo, vocab_sbo = padded_everygram_pipeline(n, train_sentences)

lm_mle = MLE(n)
lm_lpc = Laplace(n)
lm_sbo = StupidBackoff(alpha=0.4, order=n)


lm_mle.fit(train_data_mle, vocab_mle)
lm_lpc.fit(train_data_lpc, vocab_lpc)
lm_sbo.fit(train_data_sbo, vocab_sbo)

n_words = 10
random_seed = 42

print("Generated (MLE):", " ".join(lm_mle.generate(n_words, text_seed=['<s>'], random_seed=random_seed)))
print("Generated (Laplace):", " ".join(lm_lpc.generate(n_words, text_seed=['<s>'], random_seed=random_seed)))
print("Generated (StupidBackoff):", " ".join(lm_sbo.generate(n_words, text_seed=['<s>'], random_seed=random_seed)))

Generated (MLE): <s> " I ask you to the Pequod , I
Generated (Laplace): <s> " I beg your pardon Sir , come ,
Generated (StupidBackoff): <s> " I ask you to the Pequod , I


3. Try sampling (generating sentences) from the various language models. Which combination seems better? Aid yourself with the test sentences you left earlier that do not contain errors and evaluate the perplexity of the model. Show which model has better perplexity in the report and explain why you think it is the case providing examples.

In [74]:

test_sentence = test_sentences[4]  
padded_test_sentence = list(pad_both_ends(test_sentence, n=n))  
test_ngrams = list(nltk.ngrams(padded_test_sentence, n=n))  

mle_perplexity = lm_mle.perplexity(test_ngrams)
laplace_perplexity = lm_laplace.perplexity(test_ngrams)
backoff_perplexity = lm_backoff.perplexity(test_ngrams)

print(f"Test Sentence: {' '.join(test_sentence)}")
print(f"Perplexity (MLE): {mle_perplexity:.2f}")
print(f"Perplexity (Laplace): {laplace_perplexity:.2f}")
print(f"Perplexity (Backoff): {backoff_perplexity:.2f}")

Test Sentence: <s> She was the youngest of the two daughters of a most affectionate , indulgent father ; and had , in consequence of her sister ' s marriage , been mistress of his house from a very early period . </s>
Perplexity (MLE): inf
Perplexity (Laplace): 51134.00
Perplexity (Backoff): 142.26


4. The spelling corrector we want to implement follows the Noisy Channel Model. The spelled
sentence 𝑋 = 𝑥1…𝑥𝑛 is an altered version of the intended sentence 𝑊 = 𝑤1…𝑤𝑛 passing
through a noisy channel.

In [120]:
from nltk.lm.api import LanguageModel
from nltk.metrics.distance import edit_distance
from typing import List, Dict

def levenshtein(s1: str, s2: str):
    return edit_distance(s1, s2, substitution_cost=1, transpositions=True)

# This function computes close words for a vocabulary
def compute_close_words(vocab: List[str], max_dist: int = 1):
    vocab = np.array(vocab)
    word_lengths = np.array([len(w) for w in vocab])
    dict_lengths = {}
    for l in range(min(word_lengths), max(word_lengths)+1):
        dict_lengths[l] = vocab[word_lengths==l]
    min_length = min(dict_lengths.keys())
    max_length = max(dict_lengths.keys())
    close_words = {}
    for word in tqdm(vocab):
        length = len(word)
        candidate_words = []
        d1 = max(min_length, length - max_dist)
        d2 = min(max_length, length + max_dist)
        for d in range(d1, d2+1):
            candidate_words.extend(dict_lengths[d])
        close_words[word] = [w for w in candidate_words if levenshtein(word,w) <= max_dist]
    return close_words

# This function generates candidates for a sentence
def candidates(close_words: Dict[str, List[str]], vocab: List[str], sentence: str):
    sent_x = sentence.split(" ")
    for word_x in sent_x:
        assert word_x in vocab
    candidates = [sent_x]
    for ii, word_x in enumerate(sent_x):
        for cand_w in close_words[word_x]:
            if cand_w != word_x:
                cand_sent = sent_x.copy()
                cand_sent[ii] = cand_w
                candidates.append(cand_sent)
    return candidates

# This function computes the log prior for a sentence
def log_prior(sentence_w: List[str], lm: LanguageModel):
    sentence_padded = ['<s>','<s>',] + sentence_w + ['</s>']
    num_words = len(sentence_padded)
    log_prior_w = 0.0
    for i in range(2, num_words-1): # omit </s> because likelihoods don't have it
    # Uses trigrams, adapt it if you change the n
        score = lm.logscore(sentence_padded[i], [sentence_padded[i-2], sentence_padded[i-1]])
        prior_w += score
    return log_prior_w


def main():
    # Define a sample vocabulary (or use your own)
    vocab = ["hello", "world", "this", "is", "a", "test", "sentence", "different", "word"]

    # Example sentence with potential spelling errors
    sentence = "this is a worrld sentence"
    
    # 1. Compute close words
    close_words = compute_close_words(vocab, max_dist=2)  # Increased the threshold to 2
    
    # 2. Generate candidate sentences
    candidate_sentences = candidates(close_words, vocab, sentence)
    

# Call the main function if this script is being run directly
main()


100%|██████████| 9/9 [00:00<00:00, 18183.40it/s]


AssertionError: 

In [114]:
from nltk.lm.api import LanguageModel
from nltk.metrics.distance import edit_distance
from typing import List, Dict
import numpy as np
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.corpus import gutenberg
from tqdm import tqdm

def levenshtein(s1: str, s2: str):
    return edit_distance(s1, s2, substitution_cost=1, transpositions=True)

def compute_close_words(vocab: List[str], max_dist: int = 2):  # Increased max_dist to 2
    close_words = {}
    for word in tqdm(vocab):
        close_words[word] = []
        for vocab_word in vocab:
            dist = levenshtein(word, vocab_word)
            if dist <= max_dist:  # Use larger threshold
                close_words[word].append(vocab_word)
    return close_words

def candidates(close_words: Dict[str, List[str]], vocab: List[str], sentence: str):
    sent_x = sentence.split(" ")
    for word_x in sent_x:
        if word_x not in vocab:  # Handle unknown words
            print(f"Warning: '{word_x}' is not in the vocabulary!")
            continue
    candidate_sentences = [sent_x]
    for ii, word_x in enumerate(sent_x):
        if word_x in close_words:
            for cand_w in close_words[word_x]:
                if cand_w != word_x:
                    cand_sent = sent_x.copy()
                    cand_sent[ii] = cand_w
                    candidate_sentences.append(cand_sent)
    return candidate_sentences

# This function computes the log prior for a sentence
def log_prior(sentence_w: List[str], lm: LanguageModel):
    sentence_padded = ['<s>', '<s>'] + sentence_w + ['</s>']
    num_words = len(sentence_padded)
    log_prior_w = 0.0
    for i in range(2, num_words):  # Adjusted this loop to go till the last word
        score = lm.logscore(sentence_padded[i], [sentence_padded[i-2], sentence_padded[i-1]])
        log_prior_w += score
    return log_prior_w

def main():
    # Define a sample vocabulary (or use your own)
    vocab = ["hello", "world", "this", "is", "a", "test", "sentence", "different", "word"]

    # Example sentence with potential spelling errors
    sentence = "this is a worrld sentence"
    
    # 1. Compute close words
    close_words = compute_close_words(vocab, max_dist=2)  # Increased the threshold to 2
    
    # 2. Generate candidate sentences
    candidate_sentences = candidates(close_words, vocab, sentence)
    
    # 3. Train a language model (e.g., using the Gutenberg corpus)
    train_data = gutenberg.sents('austen-sense.txt')  # Sample corpus
    train_data = [list(map(str.lower, sent)) for sent in train_data]  # Optional: convert to lowercase
    n = 3  # Trigram model
    train_data, padded_vocab = padded_everygram_pipeline(n, train_data)
    
    lm = MLE(n)  # Maximum likelihood estimation
    lm.fit(train_data, padded_vocab)
    
    # 4. Compute log priors for each candidate sentence
    best_score = float('-inf')
    best_sentence = None
    for candidate in candidate_sentences:
        # Compute log prior for the candidate sentence
        score = log_prior(candidate, lm)
        
        # Update best sentence if the score is better
        if score > best_score:
            best_score = score
            best_sentence = candidate
    
    # 5. Output the best sentence
    if best_sentence:
        print("Best candidate sentence:", ' '.join(best_sentence))
        print("Log prior score:", best_score)
    else:
        print("No valid candidate sentences found.")

# Call the main function if this script is being run directly
if __name__ == "__main__":
    main()


100%|██████████| 9/9 [00:00<00:00, 7352.69it/s]


No valid candidate sentences found.


In [121]:
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.metrics.distance import edit_distance
from nltk.tokenize import word_tokenize
from typing import List, Dict
import numpy as np
from tqdm import tqdm

# Levenshtein distance function
def levenshtein(s1: str, s2: str) -> int:
    return edit_distance(s1, s2, substitution_cost=1, transpositions=True)

# Compute close words within a Levenshtein distance
def compute_close_words(vocab: List[str], max_dist: int = 1) -> Dict[str, List[str]]:
    vocab = np.array(vocab)
    word_lengths = np.array([len(w) for w in vocab])
    dict_lengths = {}
    for l in range(min(word_lengths), max(word_lengths) + 1):
        dict_lengths[l] = vocab[word_lengths == l]
    
    min_length = min(dict_lengths.keys())
    max_length = max(dict_lengths.keys())
    close_words = {}
    
    for word in tqdm(vocab):
        length = len(word)
        candidate_words = []
        d1 = max(min_length, length - max_dist)
        d2 = min(max_length, length + max_dist)
        
        for d in range(d1, d2 + 1):
            candidate_words.extend(dict_lengths[d])
        
        close_words[word] = [w for w in candidate_words if levenshtein(word, w) <= max_dist]
    
    return close_words

# Generate candidate sentences based on close words
def candidates(close_words: Dict[str, List[str]], vocab: List[str], sentence: str) -> List[List[str]]:
    sent_x = word_tokenize(sentence)
    candidate_list = [sent_x]
    
    for ii, word_x in enumerate(sent_x):
        if word_x not in vocab:  # Only correct words not in the vocabulary
            for cand_w in close_words.get(word_x, []):
                if cand_w != word_x:
                    cand_sent = sent_x.copy()
                    cand_sent[ii] = cand_w
                    candidate_list.append(cand_sent)
    
    return candidate_list

# Compute the log prior probability for a sentence
def log_prior(sentence_w: List[str], lm: MLE) -> float:
    sentence_padded = ['<s>', '<s>'] + sentence_w + ['</s>']
    num_words = len(sentence_padded)
    log_prior_w = 0.0
    
    for i in range(2, num_words):  # Trigrams: i-2, i-1, i
        score = lm.logscore(sentence_padded[i], [sentence_padded[i-2], sentence_padded[i-1]])
        log_prior_w += score
    
    return log_prior_w

# Main function to apply the spelling correction
def noisy_channel_correction(sentence: str, close_words: Dict[str, List[str]], lm: MLE) -> str:
    vocab = set(lm.vocab)
    candidates_list = candidates(close_words, vocab, sentence)
    
    best_score = -float('inf')
    best_candidate = sentence
    
    for candidate in candidates_list:
        log_p = log_prior(candidate, lm)  # Log prior probability
        # Calculate the likelihood (assuming substitution likelihood is 0.95 for matching words and 0.05 for others)
        log_likelihood = sum([np.log(0.95) if word == candidate[i] else np.log(0.05) for i, word in enumerate(candidate)])
        total_score = log_p + log_likelihood
        
        if total_score > best_score:
            best_score = total_score
            best_candidate = ' '.join(candidate)
    
    return best_candidate

# Train a language model on the Gutenberg corpus
def train_language_model(corpus: List[List[str]]) -> MLE:
    # Prepare the data for training the language model (bigram model)
    train_data, vocab = padded_everygram_pipeline(2, corpus)
    lm = MLE(2)  # Bigram model
    lm.fit(train_data, vocab)
    return lm

# Example usage:
def main():
    # Load the Gutenberg corpus
    from nltk.corpus import gutenberg
    nltk.download('gutenberg')
    nltk.download('punkt')
    
    print("Loading corpus...")
    corpus_raw = gutenberg.sents()
    corpus = [[word.lower() for word in sent] for sent in corpus_raw if len(sent) > 2]
    print(f"Loaded {len(corpus)} sentences.")
    
    lm = train_language_model(corpus)
    
    vocab = set(word for sentence in corpus for word in sentence)
    print(f"Vocabulary size: {len(vocab)} words.")
    
    close_words = compute_close_words(vocab, max_dist=2)
    print("Close words computation complete.")
    
    sentence = "this is a tast"
    print(f"Correcting sentence: '{sentence}'")
    corrected = noisy_channel_correction(sentence, close_words, lm)
    print("Corrected:", corrected)

# Run the main function
if __name__ == "__main__":
    main()


[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/macbook/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /Users/macbook/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading corpus...
Loaded 93749 sentences.
Vocabulary size: 42209 words.


TypeError: iteration over a 0-d array

### Graded Exercise 2 (3 points)

In [104]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt_tab')
from nltk.corpus import gutenberg as corpus
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
from nltk.lm import MLE, Laplace, StupidBackoff

words = corpus.words()
print(words[:20])
sentences = corpus.sents()
print(sentences[0])

training_sentences = sentences[50:]
test_sentences = sentences[:50]

n = 2
padded_sentences = [list(pad_both_ends(sent, n=n)) for sent in training_sentences]
train_mle, vocab_mle = padded_everygram_pipeline(n, padded_sentences)
train_lap, vocab_lap = padded_everygram_pipeline(n, padded_sentences)
train_back, vocab_back = padded_everygram_pipeline(n, padded_sentences)

lm_mle = MLE(n)
lm_mle.fit(train_mle, vocab_mle)

lm_laplace = Laplace(n)
lm_laplace.fit(train_lap, vocab_lap)

lm_backoff = StupidBackoff(alpha=0.4, order=n)
lm_backoff.fit(train_back, vocab_back)

n_words = 10
random_seed = 42

print("Generated (MLE):     ", " ".join(lm_mle.generate(n_words, text_seed=['<s>'], random_seed=random_seed)))
print("Generated (Laplace): ", " ".join(lm_laplace.generate(n_words, text_seed=['<s>'], random_seed=random_seed)))
print("Generated (Backoff): ", " ".join(lm_backoff.generate(n_words, text_seed=['<s>'], random_seed=random_seed)))

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/macbook/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/macbook/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', 'VOLUME', 'I', 'CHAPTER', 'I', 'Emma', 'Woodhouse', ',', 'handsome', ',', 'clever', ',', 'and', 'rich']
['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']']
Generated (MLE):      <s> " I cannot reason then thou and ideal ,
Generated (Laplace):  <s> " I cannot receive power to Sodom and David
Generated (Backoff):  <s> " I cannot reason then thou and ideal ,


### Graded Excersize 3 (1 point)
Suppose that you want to compute the Perplexity metric for a given Neural Network-based
model. You want to do so on the string
Joseph was an elderly, nay, an old man, very old, perhaps, though hale
and sinewy.
The model is autoregressive, meaning that it is trained to produce a new token 𝑤𝑡 conditioned
on 𝑤1…𝑤𝑡−1. Explain how you would do so. How different is it from computing the same
metric on an 𝑛-gram model?